In [9]:
%reload_ext autoreload
%autoreload 2
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

import spacy
from extract_features import extract_features, split_gold_column
from read_conllu_to_csv import read_conllu_write_csv
import numpy as np

In [10]:
train_file = "../UP-1.0/input/en_ewt-up-train-small.conllu"
dev_file = "../UP-1.0/input/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/input/en_ewt-up-test.conllu"

out_train_file = '../UP-1.0/output/read_conllu_train-small.csv'
out_test_file = '../UP-1.0/output/read_conllu_test.csv'
out_dev_file = '../UP-1.0/output/read_conllu_dev.csv'

feat_train_file = '../UP-1.0/features/extract_features_out_train-small.csv'
feat_test_file = '../UP-1.0/features/extract_features_out_test.csv'
feat_dev_file = '../UP-1.0/features/extract_features_out_dev.csv'

In [11]:
read_conllu_write_csv(dev_file, out_dev_file)
read_conllu_write_csv(test_file, out_test_file)
read_conllu_write_csv(train_file, out_train_file)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


File read_conllu_dev.csv produced!
File read_conllu_test.csv produced!
File read_conllu_train-small.csv produced!


,0,1,2,3,4,5,6,7,8,9,10,11
0,1,Al,Al,PROPN,NNP,Number=Sing,0,root,0:root,SpaceAfter=No,_,_
1,2,-,-,PUNCT,HYPH,_,1,punct,1:punct,SpaceAfter=No,_,_
2,3,Zaman,Zaman,PROPN,NNP,Number=Sing,1,flat,1:flat,_,_,_
3,4,:,:,PUNCT,:,_,1,punct,1:punct,_,_,_
4,5,American,american,ADJ,JJ,Degree=Pos,6,amod,6:amod,_,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...
143887,20,miseries,misery,NOUN,NNS,Number=Plur,11,obl,11:obl:for,_,_,ARG2
143888,21,of,of,ADP,IN,_,23,case,23:case,_,_,_
143889,22,the,the,DET,DT,Definite=Def|PronType=Art,23,det,23:det,_,_,_
143890,23,region,region,NOUN,NN,Number=Sing,20,nmod,20:nmod:of,SpaceAfter=No,_,_


In [4]:
extract_features(out_dev_file, feat_dev_file)
extract_features(out_train_file, feat_train_file)
extract_features(out_test_file, feat_test_file)

In [12]:
vectorizer_id = DictVectorizer()

In [13]:
def add_indetification_labels(df):
    # df = pd.read_csv( '../UP-1.0/features/extract_features_out_dev.csv', sep='\t')
    preds, args = split_gold_column(df.Label.tolist())
    ident_args = [1 if a != '_' else 0 for a in args]
    df['Identify args'] = ident_args
    df['Preds'] = preds
    return df

In [51]:
df_train = pd.read_csv(feat_train_file, sep='\t')
df_dev = pd.read_csv(feat_dev_file, sep='\t')
df_test = pd.read_csv(feat_test_file, sep='\t')

df_train = add_indetification_labels(df_train)
df_dev = add_indetification_labels(df_dev)
df_test = add_indetification_labels(df_test)

# this is used to remove the first line of each dataframe, since it takes the first line (,0,1,2,3,4,5,6,7,8,9,10,11)
# of the output of read_conllu_to_csv. 
df_train = df_train.iloc[1:]
df_dev = df_dev.iloc[1:]
df_test = df_test.iloc[1:]

# only the rows where token is not NaN.  
df_train = df_train[df_train['Token'].notna()]
df_dev = df_dev[df_dev['Token'].notna()]
df_test = df_test[df_test['Token'].notna()]

# in the label-column, only missing values are now labeled as None_placeholder.  
df_train.Label = df_train['Label'].fillna(value='None_placeholder')
df_dev.Label = df_dev['Label'].fillna(value='None_placeholder')
df_test.Label = df_test['Label'].fillna(value='None_placeholder')

#in the head columns, missing values in Head are now labeled as '_'. 
df_train.Head = df_train.Head.fillna(value='_')
df_dev.Head = df_dev.Head.fillna(value='_')
df_test.Head = df_test.Head.fillna(value='_')
#print(df_train.head())

# this drops the GOLD-columns in each file
df_train.drop(columns=['Gold'], inplace=True)
df_dev.drop(columns=['Gold'], inplace=True)
df_test.drop(columns=['Gold'], inplace=True)

df_train.drop(columns=['Label'], inplace=True)
df_dev.drop(columns=['Label'], inplace=True)
df_test.drop(columns=['Label'], inplace=True)

#in the Morphological Feature column, missing values are now labeled as '_'. 
df_train['Morphological Feature'] = df_train['Morphological Feature'].fillna(value='_')
df_dev['Morphological Feature'] = df_dev['Morphological Feature'].fillna(value='_')
df_test['Morphological Feature'] = df_test['Morphological Feature'].fillna(value='_')

df_train.to_csv('training.csv', sep='\t', index=False)
print(df_train.head())
#this checks if there are any nan-values left.
df_train.isna().sum()

      Token    PoS     Lemma Dependency    Head Head_POS  \
1        Al  PROPN        Al   compound   Zaman    PROPN   
2         -  PUNCT         -      punct   Zaman    PROPN   
3     Zaman  PROPN     Zaman       ROOT   Zaman    PROPN   
4         :  PUNCT         :      punct  killed     VERB   
5  American    ADJ  american       amod  forces     NOUN   

  Morphological Feature                                    Word Embeddings  \
1           Number=Sing  tensor([-0.7131, -0.9817,  1.2258,  0.0932, -0...   
2        PunctType=Dash  tensor([-0.9273, -1.1841,  4.7078, -0.0548,  1...   
3           Number=Sing  tensor([-0.5911, -1.4489, -0.7058,  0.7999,  0...   
4                     _  tensor([ 0.4395,  0.0779,  0.9455, -0.2140,  0...   
5            Degree=Pos  tensor([-0.8749, -0.9839, -0.6887,  1.8970, -0...   

  Named Entities                Path to head texts         Path to head POS  \
1         PERSON                            ['Al']                ['PROPN']   
2         PE

Token                    0
PoS                      0
Lemma                    0
Dependency               0
Head                     0
Head_POS                 0
Morphological Feature    0
Word Embeddings          0
Named Entities           0
Path to head texts       0
Path to head POS         0
Context                  0
POS Context              0
Identify args            0
Preds                    0
dtype: int64

## Identification model

In [16]:
X_train = df_train.drop('Identify args', axis='columns')
X_dev = df_dev.drop('Identify args', axis='columns')
X_test = df_test.drop('Identify args', axis='columns')

y_train_id = df_train['Identify args']
y_test_id = df_test['Identify args']
y_dev_id = df_dev['Identify args']

X_train_vec = vectorizer_id.fit_transform(X_train.to_dict('records'))
X_dev_vec = vectorizer_id.transform(X_dev.to_dict('records'))
X_test_vec = vectorizer_id.transform(X_test.to_dict('records'))

In [52]:
model_id = LogisticRegression(solver='liblinear', max_iter=1000)

model_id.fit(X_train_vec, y_train_id)

LogisticRegression(max_iter=1000, solver='liblinear')

In [53]:
y_pred_id = model_id.predict(X_test_vec)
#this one is executed to eventually save the predictions as a feature for classification
y_train_pred_id = model_id.predict(X_train_vec)

df_test_pred = pd.DataFrame(y_pred_id, columns=['Predicted'])
df_test_pred = pd.concat([df_test, df_test_pred], axis=1)


#this is dropped to make the output more readable
df_test_pred.drop(columns=['Word Embeddings'], inplace=True)
df_test_pred.to_csv('../UP-1.0/predictions/identification_test.tsv', sep='\t', index=False)

In [20]:
print("METRICS FOR ARG IDENTIFICATION")
print('precision', precision_score(y_test_id, y_pred_id, average='macro'), sep='\t')
print('recall', recall_score(y_test_id, y_pred_id, average='macro'), sep='\t')
print('f1', f1_score(y_test_id, y_pred_id, average='macro'), sep='\t')

METRICS FOR ARG IDENTIFICATION
precision	0.6473609012186728
recall	0.5554077674006237
f1	0.57201181977585


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test_id, y_pred_id))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95     92983
           1       0.37      0.13      0.20      9124

    accuracy                           0.90    102107
   macro avg       0.65      0.56      0.57    102107
weighted avg       0.87      0.90      0.88    102107



## Classification model

In [22]:
vectorizer = DictVectorizer()

In [56]:
df_train = pd.read_csv(feat_train_file, sep='\t')
df_dev = pd.read_csv(feat_dev_file, sep='\t')
df_test = pd.read_csv(feat_test_file, sep='\t')

#thi is from the identification, it is now made a feature 
#df_test_pred = pd.DataFrame(y_pred_id, columns=['Classification'])
#y_train_pred_id = model_id.predict(X_train_vec)
#df_train['Predicted Preds'] = y_train_pred_id

df_train = df_train.iloc[1:]
df_dev = df_dev.iloc[1:]
df_test = df_test.iloc[1:]

#the output of the identification model is here given as an input feature for the classification model
df_train['Predicted'] = y_train_pred_id
df_test['Predicted'] = y_pred_id

df_train = df_train.drop('Gold', axis='columns')
df_test = df_test.drop('Gold', axis='columns')
df_dev = df_dev.drop('Gold', axis='columns')

df_train['Label'] = df_train['Label'].replace('V', '_')
df_test['Label'] = df_test['Label'].replace('V', '_')
df_dev['Label'] = df_dev['Label'].replace('V', '_')

df_train['Morphological Feature'] = df_train['Morphological Feature'].fillna(value='_')
df_dev['Morphological Feature'] = df_dev['Morphological Feature'].fillna(value='_')
df_test['Morphological Feature'] = df_test['Morphological Feature'].fillna(value='_')

y_train = df_train['Label']
y_test = df_test['Label']
y_dev = df_dev['Label']

X_train = df_train.drop('Label', axis='columns')
X_dev = df_dev.drop('Label', axis='columns')
X_test = df_test.drop('Label', axis='columns')

df_train.isna().sum()

Token                    0
PoS                      0
Lemma                    0
Dependency               0
Head                     0
Head_POS                 0
Morphological Feature    0
Named Entities           0
Path to head texts       0
Path to head POS         0
Context                  0
POS Context              0
Label                    0
Predicted                0
dtype: int64

In [57]:
X_train_vec = vectorizer.fit_transform(X_train.to_dict('records'))
X_dev_vec = vectorizer.transform(X_dev.to_dict('records'))
X_test_vec = vectorizer.transform(X_test.to_dict('records'))

In [58]:
model = LogisticRegression(solver='liblinear', max_iter=1000)

model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [63]:
y_pred = model.predict(X_test_vec)

#this is only dropped to make the file more readable
df_test_pred = pd.DataFrame(y_pred, columns=['Pred_Classification'])
df_test_pred = pd.concat([df_test, df_test_pred], axis=1)


#this is dropped to make the output more readable
df_test_pred.drop(columns=['Word Embeddings'], inplace=True)
df_test_pred.to_csv('../UP-1.0/predictions/classification_test.tsv', sep='\t', index=False)


In [60]:
print("METRICS FOR ARG CLASSIFICATION")
print('precision', precision_score(y_test, y_pred, average='macro'), sep='\t')
print('recall', recall_score(y_test, y_pred, average='macro'), sep='\t')
print('f1', f1_score(y_test, y_pred, average='macro'), sep='\t')

METRICS FOR ARG CLASSIFICATION


/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision	0.17556101275703556


/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


recall	0.10675632935777514
f1	0.11298906665207491


In [61]:
print(classification_report(y_test, y_pred))

/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

              precision    recall  f1-score   support

          11       0.00      0.00      0.00        39
        ARG0       0.10      0.14      0.12       787
        ARG1       0.07      0.02      0.03      2735
    ARG1-DSP       0.00      0.00      0.00         0
        ARG2       0.40      0.23      0.29      1385
        ARG3       0.00      0.00      0.00        60
        ARG4       0.00      0.00      0.00        86
    ARGM-ADJ       0.00      0.00      0.00       223
    ARGM-ADV       0.13      0.01      0.02      1925
    ARGM-CAU       0.00      0.00      0.00        11
    ARGM-DIR       0.00      0.00      0.00         3
    ARGM-DIS       0.39      0.28      0.33       279
    ARGM-EXT       0.00      0.00      0.00        49
    ARGM-GOL       1.00      1.00      1.00        82
    ARGM-LOC       0.00      0.00      0.00       183
    ARGM-LVB       0.00      0.00      0.00         0
    ARGM-MNR       0.00      0.00      0.00        44
    ARGM-MOD       1.00    

/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
